In [1]:
%pip install neo4j
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase as graphdb
import sys
sys.path.append('../trengx')
import graph

In [2]:
load_dotenv()  # Load environment variables from .env file
uri = os.getenv("uri")  # Get the value of the uri variable
user = os.getenv("user")  # Get the value of the user variable
password = os.getenv("password")  # Get the value of the pw variable
g = graph.Graph(graphdb, uri, user, password)


In [4]:
g.add_node ('op', 'name', '*')

[{'node_added': '*'}]

In [5]:
g.delete_node ('op', 'name', '*')

[{'status': 'deleted'}]

In [4]:
# test chain operation
# first add then multiply

g.add_node ('num', 'name', 'x')
g.set_node_prop('name', 'x', 'value', 3.6, False)
g.add_node ('num', 'name', 'y')
g.set_node_prop('name', 'y', 'value', 1.4, False)
g.add_node ('num', 'name', 'z')
g.set_node_prop('name', 'z', 'value', 2.0, False)
g.add_node ('op', 'name', '-')
g.add_node ('num', 'name', 't')
g.set_node_prop('name', 't', 'value', 2.0, False)
g.add_node ('num', 'name', 's')
g.set_node_prop('name', 's', 'value', 0.0, False)
g.add_node ('op', 'name', '*')

g.add_edge('num2op', 'name', 'x', 'name', '-')
g.add_edge('num2op', 'name', 'y', 'name', '-')
g.add_edge('op2num', 'name', '-', 'name', 'z')
g.add_edge('num2op', 'name', 'z', 'name', '*')
g.add_edge('num2op', 'name', 't', 'name', '*')
g.add_edge('op2num', 'name', '*', 'name', 's')

g.set_node_prop('name', 'x', 'value', 3, True)

{'name': '*'}

In [9]:
# test math operation with single input
g.add_node ('num', 'name', 'x')
g.set_node_prop('name', 'x', 'value', 3.6, False)
g.add_node ('op', 'name', 'log')
g.add_edge('num2op', 'name', 'x', 'name', 'log')
g.add_node ('num', 'name', 's')
g.set_node_prop('name', 's', 'value', 0.0, False)
g.add_edge('op2num', 'name', 'log', 'name', 's')


g.set_node_prop('name', 'x', 'value', 27, True)

s
3.295836866004329


[['name', 'x', 'value', 27], ('s', 3.295836866004329)]

In [10]:
import math
math.log(27)

3.295836866004329